In [1]:
# pip install openai

In [2]:
import getpass
import requests

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length, lower, regexp_extract
from pyspark.sql.functions import to_date, when, datediff
from pyspark.sql.functions import udf  
from pyspark.sql.types import StringType, ArrayType

In [ ]:
username = ''
password = ''

In [4]:
# create access token for our login

r = requests.post('https://ssoprod.sbdinc.com/as/token.oauth2', 
data={"client_id": 'Snowflake',
"grant_type": 'password',
'username': username,
'password': password,
'client_secret': 'f9sq630wmLP6UjpSsOk7kTuP6xccCrSOC4YhE1VdTq3GCupqR7gjYcpuhEGRJ9e0',
'scope': 'session:role-any'})

r.raise_for_status()

access_token = r.json()['access_token']

# Reading snowflake table as dataframe

  
spark = SparkSession.builder.getOrCreate()

spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")


# Change the configuration based on your snowflake schema
df = spark.read.format("net.snowflake.spark.snowflake")\
.option("sfurl","sbd_caspian.us-east-1.snowflakecomputing.com")\
.option("sfUser", username)\
.option("sfDatabase","DEV_AIDA")\
.option("sfSchema","PULSE")\
.option("sfRole","YOUR_ROLE")\
.option('sfauthenticator','oauth')\
.option('autopushdown','off')\
.option("sfToken",access_token)\
.option("sfWarehouse","DEV_AIDA_WH")\
.option("query", """SELECT *
        FROM REVIEWS_MASTER;""")\
.load()

In [7]:
clean_reviews = 'this is a good product. i love this drill it knows how to make holes in anything. Not really my thing. I prefer other brands over DeWalt. Dewalt aka QUALITY.'

In [ ]:
#Note: The openai-python library support for Azure OpenAI is in preview.
#Note: This code sample requires OpenAI Python library version 1.0.0 or higher.
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = "https://openai-sbd-poc.openai.azure.com/", 
  api_key='3a81d76d39114af784d0e2b116af0c0f',  
  api_version="2024-02-15-preview"
)

message_text = [{"role":"system","content":"You are an E-Commerce expert!"},
               {"role": "user", "content": f"Summarise these reviews: {clean_reviews}"}]

completion = client.chat.completions.create(
  model="SBD-PrivateChatGPT", # model = "deployment_name"
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

In [19]:
choices = completion.choices

In [20]:
message_content = choices[0].message.content  

In [21]:
message_content

"The reviews present a mixed sentiment about the product. One reviewer is highly satisfied, praising the drill's effectiveness in making holes in various materials. Another expresses a preference for other brands over DeWalt, suggesting dissatisfaction or a better experience with competitors. Meanwhile, a third review equates DeWalt with quality, indicating a positive perception of the brand. Overall, the reviews suggest that while some customers are pleased with the product's performance and associate the brand with high quality, others may not favor it as much and prefer alternatives."